In [72]:
import scipy.interpolate
import astropy.table
import numpy
import time
import glob

import numpy as np
import numpy.random as random
import statistics
import matplotlib.pyplot as plt
from matplotlib import colors

import os

import pandas as pd

In [21]:
def read_pms_data(tracks="BHAC15"):

    # Load in the data for the appropriate set of evolutionary tracks.

    path = './data/evolutionary_tracks/'
    
    if tracks == "BHAC15":
        f = open(path+"BHAC15_tracks+structure.txt","r")
        lines = f.readlines()
        f.close()

        colnames = lines[46].split()[1:]

        data = numpy.loadtxt(path+"BHAC15_tracks+structure.txt", comments="!", \
                skiprows=45)
    elif tracks == "Siess2000":
        f = open(path+"siess_2000/m0.13z02.hrd")
        lines = f.readlines()
        f.close()

        line1 = lines[0].replace(" (","_(").replace("log g","logg").\
                replace("#","").replace("_(Lo)","/Ls").replace("age_","log_t").\
                split()
        line2 = lines[1].replace(" (","_(").replace("log g","logg").\
                replace("#","").replace("_(Mo)","/Ms").split()

        colnames = line1 + line2
        colnames[0::2] = line1
        colnames[1::2] = line2

        files = glob.glob(path+"siess_2000/*.hrd")
        for file in files:
            try:
                data = numpy.concatenate((data, numpy.loadtxt(file)))
            except:
                data = numpy.loadtxt(file)

        # Fix the stellar luminosity.

        data[:,2] = numpy.log10(data[:,2])
        data[:,-1] = numpy.log10(data[:,-1])
    elif tracks == "Dotter2008":
        f = open(path+"dotter2008/m200fehp00afep0.trk")
        lines = f.readlines()
        f.close()

        # Get the column names.

        colnames = ['M/Ms'] + lines[1].replace("Log ","Log").\
                replace("Age ","log_t").replace("LogT","Teff").\
                replace("LogL","L/Ls").replace("yrs","yr").split()[1:]

        # Now read in the data files.

        files = glob.glob(path+"dotter2008/*.trk")
        for file in files:
            new_data = numpy.loadtxt(file)

            # add a column with the mass of the star.

            mass = float(file.split("/")[-1][1:4])/100.
            mass_arr = numpy.zeros((new_data.shape[0],1)) + mass
            new_data = numpy.hstack((mass_arr, new_data))

            # Merge with existing data.

            try:
                data = numpy.concatenate((data, new_data))
            except:
                data = new_data.copy()

        # Get rid of ages more than ~50Myr.

        good = data[:,1] < 50.0e6
        data = data[good,:]

        # Fix some of the columns.

        data[:,1] = numpy.log10(data[:,1])
        data[:,2] = 10.**data[:,2]

    elif tracks == "Tognelli2011":
        f = open(path+"tognelli2011/Z0.02000_Y0.2700_XD2E5_ML1.68_AS05/"
                "TRK_M0.20_Z0.02000_Y0.2700_XD2E5_ML1.68_AS05.DAT")
        lines = f.readlines()
        f.close()

        # Get the column names.

        colnames = ['M/Ms'] + lines[3].replace("LOG ","LOG_").\
                replace("LOG_AGE","log_t(yr)").replace("LOG_L","L/Ls").\
                replace("LOG_TE","Teff").split()[1:]

        # Now read in the data files.

        files = glob.glob(path+"tognelli2011/"
                "Z0.02000_Y0.2700_XD2E5_ML1.68_AS05/*.DAT")
        for file in files:
            new_data = numpy.loadtxt(file)

            # add a column with the mass of the star.

            mass = float(file.split("/")[-1].split("_")[1][1:])
            mass_arr = numpy.zeros((new_data.shape[0],1)) + mass
            new_data = numpy.hstack((mass_arr, new_data))

            # Merge with existing data.

            try:
                data = numpy.concatenate((data, new_data))
            except:
                data = new_data.copy()

        # Fix some of the columns.

        data[:,5] = 10.**data[:,5]

    elif tracks == "Feiden2016":
        f = open(path+"feiden2016/std/"
                "m0090_GS98_p000_p0_y28_mlt1.884.trk")
        lines = f.readlines()
        f.close()

        # Get the column names.

        colnames = ['M/Ms'] + lines[3].replace("Log ","Log").\
                replace("Age ","log_t").replace("LogT","Teff").\
                replace("LogL","L/Ls").replace("yrs","yr").split()[1:6]

        # Now read in the data files.

        files = glob.glob(path+"feiden2016/std/*.trk")
        for file in files:
            new_data = numpy.loadtxt(file, usecols=(0,1,2,3,4))

            # add a column with the mass of the star.

            mass = float(file.split("/")[-1][1:5])/1000.
            mass_arr = numpy.zeros((new_data.shape[0],1)) + mass
            new_data = numpy.hstack((mass_arr, new_data))

            # Merge with existing data.

            try:
                data = numpy.concatenate((data, new_data))
            except:
                data = new_data.copy()

        # Get rid of ages more than ~50Myr.

        good = data[:,1] < 50.0e6
        data = data[good,:]

        # Fix some of the columns.

        data[:,1] = numpy.log10(data[:,1])
        data[:,2] = 10.**data[:,2]

    elif tracks == "Feiden2016mag":
        f = open(path+"feiden2016/mag/"
                "m1700_GS98_p000_p0_y28_mlt1.884_mag08kG.ntrk")
        lines = f.readlines()
        f.close()

        # Get the column names.

        colnames = ['M/Ms'] + lines[8].replace("conv. ","conv.").\
                replace("AGE","log_t(yr)").replace("log(Teff)","Teff").\
                replace("log(L/Lsun)","L/Ls").replace("Model #","Model#").\
                replace("M He core","M_He_core").replace(",","").split()[1:]

        # Now read in the data files.

        files = glob.glob(path+"feiden2016/mag/*.ntrk")
        for file in files:
            new_data = numpy.loadtxt(file,usecols=tuple([i for i in range(12)]))

            # add a column with the mass of the star.

            mass = float(file.split("/")[-1][1:5])/1000.
            mass_arr = numpy.zeros((new_data.shape[0],1)) + mass
            new_data = numpy.hstack((mass_arr, new_data))

            # Merge with existing data.

            try:
                data = numpy.concatenate((data, new_data))
            except:
                data = new_data.copy()

        # Fix some of the columns.

        data[:,3] = numpy.log10(data[:,3]*1.0e9)
        data[:,7] = 10.**data[:,7]

    elif tracks == "Chen2014":
        f = open(path+"bressan2012/Z0.017Y0.279/"
                "Z0.017Y0.279OUTA1.77_F7_M000.700.DAT")
        lines = f.readlines()
        f.close()

        # Get the column names.

        colnames = lines[0].replace("LOG ","LOG_").\
                replace("AGE","log_t(yr)").replace("LOG_L","L/Ls").\
                replace("LOG_TE","Teff").replace("MASS","M/Ms").split()

        # Now read in the data files.

        files = glob.glob(path+"bressan2012/Z0.017Y0.279/*.DAT")

        for file in files:
            new_data = numpy.loadtxt(file, skiprows=2)

            # Merge with existing data.

            try:
                data = numpy.concatenate((data, new_data))
            except:
                data = new_data.copy()

        # Get rid of ages more than ~50Myr.

        good = data[:,2] < 50.0e6
        data = data[good,:]

        # Fix some of the columns.

        data[:,2] = numpy.log10(data[:,2])
        data[:,4] = 10.**data[:,4]

    elif tracks == "Bressan2012":
        f = open(path+"bressan2012/bressan2012.dat")
        lines = f.readlines()
        f.close()

        # Get the column names.

        colnames = lines[13].replace("log(age/yr)","log_t(yr)").\
                replace("logL/Lo","L/Ls").replace("logTe","Teff").\
                replace("M_act","M/Ms").split()[1:]

        # Now read in the data files.

        data = numpy.loadtxt(path+"bressan2012/bressan2012.dat")

        # Fix some of the columns.

        data[:,5] = 10.**data[:,5]

    # Make the data into a table.

    table = astropy.table.Table(data, names=colnames)

    # Return the table now.

    return table


def pms_get_mstar(age=None, luminosity=None, tracks="BHAC15"):

    # Load in the data for the appropriate set of evolutionary tracks.

    table = read_pms_data(tracks=tracks)

    # Now do the 2D interpolation.

    Mstar = scipy.interpolate.LinearNDInterpolator((table["L/Ls"], \
            table["log_t(yr)"]), table["M/Ms"], rescale=True)

    # Finally, get the stellar mass.

    if isinstance(age,float) and isinstance(luminosity,float):
        xi = numpy.array([[luminosity, numpy.log10(age)]])
    elif isinstance(age,float):
        xi = numpy.array([[luminosity[i],numpy.log10(age)] for i in range(len(luminosity))])
    else:    
        xi = numpy.array([[luminosity[i],numpy.log10(age[i])] for i in \
                range(len(age))])

    return Mstar(xi)

def pms_get_teff(luminosity = None, age=1.0e6, tracks="BHAC15"):

    # Load in the data for the appropriate set of evolutionary tracks.

    table = read_pms_data(tracks=tracks)

    # Now do the 2D interpolation.

    Teff = scipy.interpolate.LinearNDInterpolator((table["L/Ls"], \
            table["log_t(yr)"]), table["Teff"])

    # Finally, get the stellar mass.

    if isinstance(age,float) and isinstance(luminosity,float):
        xi = numpy.array([[luminosity, numpy.log10(age)]])
    elif isinstance(age,float):
        xi = numpy.array([[luminosity[i],numpy.log10(age)] for i in range(len(mass))])
    else:
        xi = numpy.array([[luminosity[i],numpy.log10(age[i])] for i in range(len(age))])

    return Teff(xi)

def pdspy_get_teff(mass=1.0, age=1.0e6, tracks="BHAC15"):

    # Load in the data for the appropriate set of evolutionary tracks.

    table = read_pms_data(tracks=tracks)

    # Now do the 2D interpolation.

    Teff = scipy.interpolate.LinearNDInterpolator((table["M/Ms"], \
            table["log_t(yr)"]), table["Teff"])

    # Finally, get the stellar mass.
    
    if isinstance(age,float) and isinstance(mass,float):
        xi = numpy.array([[mass, numpy.log10(age)]])
    elif isinstance(age,float):
        xi = numpy.array([[mass[i],numpy.log10(age)] for i in range(len(mass))])
    else:
        xi = numpy.array([[mass,numpy.log10(age[i])] for i in range(len(age))])

    return Teff(xi)

def pdspy_get_luminosity(mass=1.0, age=1.0e6, tracks="BHAC15"):

    # Load in the data for the appropriate set of evolutionary tracks.

    table = read_pms_data(tracks=tracks)

    # Now do the 2D interpolation.

    Lstar = scipy.interpolate.LinearNDInterpolator((table["M/Ms"],\
            table["log_t(yr)"]), table["L/Ls"])

    # Finally, get the stellar mass.
    print('test')
    if isinstance(age,float) and isinstance(mass,float):
        xi = numpy.array([[mass, numpy.log10(age)]])
    elif isinstance(age,float):
        xi = numpy.array([[mass[i],numpy.log10(age)] for i in range(len(mass))])
    else:
        xi = numpy.array([[mass,numpy.log10(age[i])] for i in range(len(age))])
    print('test_end')
    return 10.**Lstar(xi)
        

In [15]:
def monte_carlo(flux=5.6e-2, D=0.4):
    b = random.normal(-2.66, 0.06, 1000)
    m = random.normal(0.91, 0.06, 1000)
    flux_al = random.normal(flux, 0.1*flux, 1000)
    L_bol_al = []

    lum = flux_al * (D**2)
    for i in range(1000):
        L_bol_al.append((np.log10(lum[i]) - b[i])/m[i])
    L_bol_mean = statistics.mean(L_bol_al)
    L_bol_std = statistics.stdev(L_bol_al)
    return L_bol_al, L_bol_mean, L_bol_std
    
def plot_hist(flux, arr, mean, typ='exponent'):
    N, bins, patches = plt.hist(arr, bins=100)
    max_index = np.argmax(N)
    max_value = bins[max_index]

    fracs = N / N.max()
    norm = colors.Normalize(fracs.min(), fracs.max())

    for thisfrac, thispatch in zip(fracs, patches):
        color = plt.cm.viridis(norm(thisfrac))
        thispatch.set_facecolor(color)
    plt.axvline(mean, color='r', linestyle='dashed', linewidth=1, label = 'average')
    plt.axvline(max_value, color='b', linestyle='dashed', linewidth=1, label = 'most frequent')
    plt.legend()
    if typ =='exponent':
        plt.xlabel('exponent of L_bol')
        plt.ylabel('frequency')
        print(f'most frequent result for {flux} mJy is: {max_value}')
    elif typ =='mass':
        plt.xlabel('mass of the object')
        plt.ylabel('frequency')
        print(f'most frequent result for {flux} mJy is: {max_value} solar mass')
    return max_value

In [16]:
def find_flux(flux, lam_in, lam_out, exp=-0.1):
    f_in = 3.0e8/lam_in
    f_out = 3.0e8/lam_out
    return flux*(f_out/f_in)**(exp)
    

In [56]:
def set_axis_style(ax, labels):
#     ax.xaxis.set_tick_params(direction='out')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xticks(np.arange(1, len(labels) + 1))
    ax.set_xticklabels(labels, rotation=45)
    ax.set_xlim(0.25, len(labels) + 0.75)
    ax.set_xlabel('Evolutionary Tracks', fontsize=16)

In [81]:
def plot_violin(mass_1y_e056, mass_1y_en01, mass_05y_e056, mass_05y_en01, name):
    fig, ((ax1,ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(15,10), sharey=True)
    ax1.set_title(f'{name}'+ r", $L_{bol} \propto \nu^{0.56}$" + r", 1 $Myr$", fontsize=20)
    ax1.set_ylabel('$M_*$ (M$_{\odot}$)', fontsize=16)
    ax1.violinplot(mass_1y_e056)
    ax2.set_title(f'{name}'+ r", $L_{bol} \propto \nu^{-0.1}$" + r", 1 $Myr$", fontsize=20)
    ax2.set_ylabel('$M_*$ (M$_{\odot}$)', fontsize=16)
    ax2.violinplot(mass_1y_en01)
    ax3.set_title(f'{name}'+ r", $L_{bol} \propto \nu^{0.56}$" + r", 0.5 $Myr$", fontsize=20)
    ax3.set_ylabel('$M_*$ (M$_{\odot}$)', fontsize=16)
    ax3.violinplot(mass_05y_e056)
    ax4.set_title(f'{name}'+ r", $L_{bol} \propto \nu^{-0.1}$" + r", 0.5 $Myr$", fontsize=20)
    ax4.set_ylabel('$M_*$ (M$_{\odot}$)', fontsize=16)
    ax4.violinplot(mass_05y_en01)

    labels = ["BHAC15","Siess2000",  "Dotter2008", "Tognelli2011","Feiden2016","Feiden2016mag","Chen2014","Bressan2012"]
    for ax in [ax1, ax2, ax3, ax4]:
        set_axis_style(ax, labels)
    fig.tight_layout()
    direc = 'pictures_violin'
    os.makedirs(direc, exist_ok=True)
    plt.savefig(os.path.join(direc, f'{name}_violin_plot.pdf'))
    plt.close()

In [82]:
def save_mass_distribution_tabel(data, file_name, direc):
    
    trac = ["BHAC15","Siess2000",  "Dotter2008", "Tognelli2011","Feiden2016","Feiden2016mag","Chen2014","Bressan2012"]

    mass_tabel = pd.DataFrame({trac[i]: data[i]  for i in range(8)})
    os.makedirs(direc, exist_ok=True)  
    mass_tabel.to_csv(os.path.join(direc, f'{file_name}.csv'))
    return mass_tabel

In [83]:
# Disks information in dictionary
obj_56 = {'name': 'HOPS-56', 'frequency': 33.0e9, 'flux':5.69e-5}
obj_65 = {'name': 'HOPS-65', 'frequency': 15.0e9, 'flux':1.064e-5}
obj_124 = {'name': 'HOPS-124', 'frequency': 44.0e9, 'flux': 0.0005424}
obj_140 = {'name': 'HOPS-140', 'frequency': 33.0e9, 'flux':0.0001537}
obj_157_a = {'name': 'HOPS-157_a', 'frequency': 33.0e9, 'flux':2.67e-5}
obj_157_b = {'name': 'HOPS-157_b', 'frequency': 33.0e9, 'flux':2.34e-5}
obj_163_3RMS = {'name': 'HOPS-163', 'frequency': 33.0e9, 'flux':4.87e-6*3}
obj_270_3RMS = {'name': 'HH270MMS2', 'frequency': 44.0e9, 'flux':8.75e-6*3}

obj_56_3RMS = {'name': 'HOPS-56', 'frequency': 33.0e9, 'flux':7.81e-6*3}
obj_65_3RMS = {'name': 'HOPS-65', 'frequency': 15.0e9, 'flux':2.44e-6*3}
obj_124_3RMS = {'name': 'HOPS-124', 'frequency': 44.0e9, 'flux': 6.54e-5*3}
obj_140_3RMS = {'name': 'HOPS-140', 'frequency': 33.0e9, 'flux':7.54e-6*3}
obj_157_3RMS = {'name': 'HOPS-157', 'frequency': 33.0e9, 'flux':9.49e-6*3}


objs = [obj_56, obj_56_3RMS, obj_65, obj_65_3RMS, obj_124, obj_56_3RMS, obj_140, obj_140_3RMS, obj_157_a, obj_157_b, obj_157_3RMS, obj_163_3RMS, obj_270_3RMS]


for obj in objs:
    print(obj)
    
    lam_in = 3.0e8/(obj['frequency'])
    lam_out = 4.1e-2
    fluxin = obj['flux']
    flux_41_01 = find_flux(fluxin*1000, lam_in, lam_out, -0.1)
    flux_41_051 = find_flux(fluxin*1000, lam_in, lam_out, 0.51)

    trac = ["BHAC15","Siess2000",  "Dotter2008", "Tognelli2011","Feiden2016","Feiden2016mag","Chen2014","Bressan2012"]

    #sample via monte carlo method
    L_bol_056, L_mean_056, L_std_056 = monte_carlo(flux=flux_41_051, D=0.4)
    L_bol_n01, L_mean_n01, L_std_n01 = monte_carlo(flux=flux_41_01, D=0.4)

    #calculate mass distribution
    mass_al_trac_1y = np.empty(len(trac), dtype=object)
    mass_al_trac_1yn = np.empty(len(trac), dtype=object)
    mass_al_trac_05y = np.empty(len(trac), dtype=object)
    mass_al_trac_05yn = np.empty(len(trac), dtype=object)

    for track in trac: 
        mass_al_trac_1y[trac.index(track)] = pms_get_mstar(age = 1.0e6, luminosity= L_bol_056, tracks=track)
        mass_al_trac_1yn[trac.index(track)] = pms_get_mstar(age = 1.0e6, luminosity= L_bol_n01, tracks=track)
        mass_al_trac_05y[trac.index(track)] = pms_get_mstar(age = 0.5e6, luminosity=L_bol_056, tracks=track)
        mass_al_trac_05yn[trac.index(track)] = pms_get_mstar(age = 0.5e6, luminosity= L_bol_n01, tracks=track)

    # generate violin plot
    plot_violin(mass_al_trac_1y, mass_al_trac_1yn, mass_al_trac_05y, mass_al_trac_05yn, obj['name'])
    
    # create pandas dataframe and save to csv files
    save_mass_distribution_tabel(mass_al_trac_1y, 'mass_tabel_prop056_1myr', 'data_result')
    save_mass_distribution_tabel(mass_al_trac_1yn, 'mass_tabel_propn1_1myr', 'data_result')
    save_mass_distribution_tabel(mass_al_trac_05y, 'mass_tabel_prop056_05myr', 'data_result')
    save_mass_distribution_tabel(mass_al_trac_05yn, 'mass_tabel_propn1_05myr', 'data_result')
    

{'name': 'HOPS-56', 'frequency': 33000000000.0, 'flux': 5.69e-05}
{'name': 'HOPS-56', 'frequency': 33000000000.0, 'flux': 2.3429999999999998e-05}
{'name': 'HOPS-65', 'frequency': 15000000000.0, 'flux': 1.064e-05}
{'name': 'HOPS-65', 'frequency': 15000000000.0, 'flux': 7.319999999999999e-06}
{'name': 'HOPS-124', 'frequency': 44000000000.0, 'flux': 0.0005424}
{'name': 'HOPS-56', 'frequency': 33000000000.0, 'flux': 2.3429999999999998e-05}
{'name': 'HOPS-140', 'frequency': 33000000000.0, 'flux': 0.0001537}
{'name': 'HOPS-140', 'frequency': 33000000000.0, 'flux': 2.262e-05}
{'name': 'HOPS-157_a', 'frequency': 33000000000.0, 'flux': 2.67e-05}
{'name': 'HOPS-157_b', 'frequency': 33000000000.0, 'flux': 2.34e-05}
{'name': 'HOPS-157', 'frequency': 33000000000.0, 'flux': 2.8470000000000004e-05}
{'name': 'HOPS-163', 'frequency': 33000000000.0, 'flux': 1.461e-05}
{'name': 'HH270MMS2', 'frequency': 44000000000.0, 'flux': 2.6249999999999998e-05}
